In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
import util
from model import Generator, Discriminator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
tf.keras.backend.set_floatx('float32')
from sklearn.impute import SimpleImputer

In [ ]:
from numba import cuda
os.environ['CUDA_VISIBLE_DEVICES']="0"
print(tf.test.is_gpu_available())

In [ ]:
df=pd.read_csv("./dataset/df_noOutliner_ana.csv",index_col=0)

In [ ]:
df, imp_mode, imp_mean=util.FeatureArrange(df)
df.shape

In [ ]:
sc = MinMaxScaler()
df = sc.fit_transform(df)
df.shape

In [ ]:
## setting hyperparameter
latent_dim = 73
epochs = 1
batch_size= 128
buffer_size = 6000
# save_interval = 50
n_critic = 5

In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
gen_opt = tf.keras.optimizers.Adam(0.0001, 0.5, 0.9)
disc_opt = tf.keras.optimizers.Adam(0.0001, 0.5, 0.9)

In [ ]:
training_dataset=tf.data.Dataset.from_tensor_slices(df.astype('float32'))\
    .shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [ ]:
@tf.function
def train_step(x):
    noise = tf.random.normal([batch_size, latent_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
        gen_data = generator(noise, training=True)
        gen_output = discriminator(gen_data, training=True)
        real_output = discriminator(x, training=True )
        
#         print("gen_output:",gen_output)
#         print("real_output:",real_output)
        
        x_hat = util.random_weight_average(x, gen_data)
        d_hat = discriminator(x_hat,)
#         print("x_hat:",x_hat)
#         print("d_hat:",d_hat)
        
        disc_loss = util.discriminator_loss(real_output, gen_output, d_hat, x_hat)
#         print("disc_loss:",disc_loss)
        gen_loss = util.generator_loss(gen_output)
#         print("gen_loss:",gen_loss)
        
    grad_disc = dis_tape.gradient(disc_loss, discriminator.trainable_variables)
    grad_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    
    disc_opt.apply_gradients(zip(grad_disc, discriminator.trainable_variables))
    gen_opt.apply_gradients(zip(grad_gen, generator.trainable_variables))
    return gen_output, real_output,x_hat,d_hat

In [ ]:
@tf.function
def train_discriminator(x):
    
    noise = tf.random.normal([x.shape[0], latent_dim])

    with tf.GradientTape() as dis_tape:
        gen_data = generator(noise, training=True)
#         print("gen_data",gen_data)
        gen_output = discriminator(gen_data,training=True)
#         print("gen_output",gen_output)
        real_output = discriminator(x, training=True)
#         print("real_output",real_output)
        x_hat = util.random_weight_average(x, gen_data)
        d_hat = discriminator(x_hat,)
        
        disc_loss = util.discriminator_loss(real_output, gen_output, d_hat, x_hat)
    grad_disc = dis_tape.gradient(disc_loss, discriminator.trainable_variables)
    disc_opt.apply_gradients(zip(grad_disc, discriminator.trainable_variables))

    return disc_loss


@tf.function
def train_generator():
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape:
        gen_data = generator(noise, training=True )
        gen_output = discriminator(gen_data,training=True)

        gen_loss = util.generator_loss(gen_output)

    grad_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gen_opt.apply_gradients(zip(grad_gen, generator.trainable_variables))

    return gen_loss

In [ ]:
for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0
    
    for data in training_dataset:
#         gen_output, real_output,x_hat,d_hat=train_step(data)
        disc_loss += train_discriminator(data)
#         print("gen_output:",gen_output)
#         print("real_output", real_output,)
#         print("x_hat", x_hat)
#         print("d_hat",d_hat)
    
        if disc_opt.iterations.numpy() % n_critic == 0:
            gen_loss += train_generator()
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1,
                                                                               time.time() - start, gen_loss / batch_size, disc_loss / (batch_size*n_critic)))

In [ ]:
# disc_opt.iterations.numpy()

In [ ]:
print(12)